# How to implement SwiftUI(iOS14+) video call with Azure Communication Services


## Intro

This article aims to show a work through of integrating ACS + ANH + CallKit + PushKit for SwiftUI iOS application. Yes right! all together at once. Sounds bit much though, you’ll get a nice working example after you go through this article.

ACS(Azure Communication Services) is fairly new to the industry(Pre release announced around Sep. 2020) and there are quite many voice chat service providers like Amazon, Twilio, and etc… But I chose ACS over other services because ACS integrates all the communication related services as one single service like connecting landline phone to the service(limited for now), text message, QnA chat bot. If you are keen to learn more about the service, please go to [communication service overview](https://azure.microsoft.com/en-us/services/communication-services/#overview).

Prerequisites([adopted from ACS quickstart tutorial](https://docs.microsoft.com/en-us/azure/communication-services/quickstarts/voice-video-calling/getting-started-with-calling?pivots=platform-ios#prerequisites) and [ANH quickstart tutorial](https://docs.microsoft.com/en-us/azure/notification-hubs/ios-sdk-get-started#prerequisites))

- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/en-us/free/?WT.mc_id=A261C142F).
- A deployed Communication Services resource. [Create a Communication Services resource](https://docs.microsoft.com/en-us/azure/communication-services/quickstarts/create-communication-resource?tabs=linux&pivots=platform-azp).
- A *User Access Token* to enable the call client. For more information on [how to get a User Access Token](https://docs.microsoft.com/en-us/azure/communication-services/quickstarts/access-tokens?tabs=linux&pivots=programming-language-javascript)
- An active [Apple Developer account](https://developer.apple.com/).
- A Mac running [Xcode](https://developer.apple.com/xcode/), along with a valid developer certificate installed into your Keychain.
- An iPhone or iPad running iOS version 10 or later.
- Your physical device registered in the [Apple Portal](https://developer.apple.com/) and associated with your certificate.

For ANH prerequisites, please take option 2 of [Create a certificate for Notification Hubs](https://docs.microsoft.com/en-us/azure/notification-hubs/ios-sdk-get-started#create-a-certificate-for-notification-hubs) as we are going to use token-based authentication due to the service change for iOS13. See more detail [here](https://docs.microsoft.com/en-us/azure/notification-hubs/push-notification-updates-ios-13).

If you are having some trouble with above prerequisites, don’t worry it’s normal. Just read their manual and don’t miss what the manual says to do so then you’ll be fine. If you are still having some trouble with configuring above, please wait for my next article where I will be explaining those configuration step by step with detailed screenshots.

If you are having some trouble with above prerequisites, don’t worry it’s normal. Just read their manual and don’t miss what the manual says to do so then you’ll be fine. If you are still having some trouble with configuring above, please wait for my next article where I will be explaining those configuration step by step with detailed screenshots.


## Project initialization

![](img/1.png)



Create a new project with Product Name and Organization Identifier. You can change these name to whatever you want.

Product Name: AzureCommunicationVideoCallingSample

Organization Identifier: com.contoso

![](img/2.png)

When press Command + Option + p, preview canvas will display what the app will look like. Hello, world! for now.

Now create a Podfile to include ACS and ANH pods.


#### [Podfile](code/Podfile)

In [ ]:
platform :ios, '13.0'
use_frameworks!
target 'AzureCommunicationVideoCallingSample' do
    pod 'AzureCommunicationCalling', '~> 1.0.0-beta.8'
    pod 'AzureNotificationHubs-iOS'
end

Install pods by ‘pod install’ command

![](img/3.png)

### Initial configuration

Now open the project as workspace.

Let’s create some plist to hold our ACS and ANH credentials.

Create DevSettings.plist, FirstUser.plist, SecondUser.plist and fill in values from above prerequisites section.

#### [DevSettings.plist](code/DevSettings.plist)

In [ ]:
<plist version="1.0">
    <dict>
        <key>HUB_NAME</key>
        <string/>
        <key>CONNECTION_STRING</key>
        <string/>
    </dict>
</plist>

#### [FirstUser.plist](code/FirstUser.plist)

In [ ]:
<plist version="1.0">
    <dict>
        <key>DISPLAYNAME</key>
        <string/>
        <key>IDENTIFIER</key>
        <string/>
        <key>TOKEN</key>
        <string/>
        <key>CALLEE</key>
        <string/>
    </dict>
</plist>

#### [SecondUser.plist](code/SecondUser.plist)

In [ ]:
<plist version="1.0">
    <dict>
        <key>DISPLAYNAME</key>
        <string/>
        <key>IDENTIFIER</key>
        <string/>
        <key>TOKEN</key>
        <string/>
        <key>CALLEE</key>
        <string/>
    </dict>
</plist>

In order to read these plist values, we are going to create some helper methods. Create a folder named Helpers and create a file named Constants.swift.

#### [Constants.swift](code/Constants.swift)

In [ ]:
import Foundation

struct Constants {
    static var hubName = ""
    static var connectionString = ""

    static var displayName = ""
    static var identifier = ""
    static var token = ""
    static var callee = ""
}

Create another file named PListHelper.swift in same folder.

#### [PListHelper.swift](code/PlistHelper.swift)

In [ ]:
import Foundation

func getPlistInfo(resourceName: String, key: String) -> String {
    guard let path = Bundle.main.path(forResource: resourceName, ofType: "plist"),
          let configValues = NSDictionary(contentsOfFile: path),
          let value = configValues[key] as? String else {
            return ""
        }

    return value
}

When you are done with above tasks, your folder structure will look like below.

Open AzureCommunicationVideoCallingSampleApp.swift and modify as below. We will read credentials and connection strings from DevSettings.plist, {First / Second}User.plist and fill in our constants for later use.

#### [AzureCommunicationVideoCallingSampleApp.swift](code/AzureCommunicationVideoCallingSampleApp.swift)

In [ ]:
import SwiftUI

@main
struct AzureCommunicationVideoCallingSampleApp: App {

    init() {
        // Fill in DevSettings.plist for AzureNotificationHubs hubName and connectionString.
        Constants.hubName = getPlistInfo(resourceName: "DevSettings", key: "HUB_NAME")
        Constants.connectionString = getPlistInfo(resourceName: "DevSettings", key: "CONNECTION_STRING")

        // Fill in FirstUser.plist with displayName, identifier, token and receiver identifier to test call feature.
        // Change resouceName to "FirstUser" or "SecondUser" to deploy different credentials.
        let resourceName = "FirstUser"
        Constants.displayName = getPlistInfo(resourceName: resourceName, key: "DISPLAYNAME")
        Constants.identifier = getPlistInfo(resourceName: resourceName, key: "IDENTIFIER")
        Constants.token = getPlistInfo(resourceName: resourceName, key: "TOKEN")
        Constants.callee = getPlistInfo(resourceName: resourceName, key: "CALLEE")
    }

    var body: some Scene {
        WindowGroup {
            ContentView()
        }
    }
}

### Tab navigations
This sample has 2 tabs. Home and Profile.

In order to create a tab navigation, we will create two views and define enum for each tab.

Create Views folder and HomeView.swift and ProfileView.swift inside the folder.

Change “Hello, World!” to tab name accordingly. for ex) “Home” and “Profile”

Next, create a folder named Enum and create Tabs.swift file with below content.

#### [Tabs.swift](code/Tabs.swift)

In [ ]:
import SwiftUI

enum Tab: Int, Identifiable, CaseIterable {
    case home
    case profile

    var id: Int {
        return rawValue
    }

    var name: String {
        switch self {
        case .home:
            return "Home"
        case .profile:
            return "Profile"
        }
    }

    private var imageName: String {
        switch self {
        case .home:
            return "list.bullet"
        case .profile:
            return "person"
        }
    }

    var tabItem: some View {
        Group {
            Text(name)
            Image(systemName: imageName)
        }
    }

    var presentingView: some View {
        switch self {
        case .home:
            return AnyView(HomeView())
        case .profile:
            return AnyView(ProfileView())
        }
    }
}

Open ContentView.swift and change as below.

#### [ContentView.swift](code/ContentView.swift)

In [ ]:
import SwiftUI

struct ContentView: View {
    @State var currentTab: Tab = .home

    var body: some View {
        NavigationView {
            TabView(selection: $currentTab) {
                ForEach(Tab.allCases) { tab in
                    tab.presentingView
                        .tabItem { tab.tabItem }
                        .tag(tab)
                }
            }
            .navigationBarTitle(currentTab.name, displayMode: .inline)
        }
    }
}

struct ContentView_Previews: PreviewProvider {
    static var previews: some View {
        ContentView()
    }
}

Authentication
Create CommunicationUserTokenModel.swift and AuthenticationViewModel.swift as below.

#### [CommunicationUserTokenModel.swif](code/CommunicationUserTokenModel.swift)

In [ ]:
import Foundation

public struct CommunicationUserTokenModel: Codable {
    public var token: String?
    public var expiresOn: Date?
    public var communicationUserId: String?

    public init(token: String? = nil, expiresOn: Date? = nil, communicationUserId: String? = nil) {
        self.token = token
        self.expiresOn = expiresOn
        self.communicationUserId = communicationUserId
    }
}

#### [AuthenticationViewModel.swift](code/AuthenticationViewModel.swift)

In [ ]:
import Combine
import AzureCommunicationCalling

class AuthenticationViewModel: ObservableObject {
    @Published var currentTab: Tab = .home
    @Published var isAuthenticating = false
    @Published var signInRequired = false
    @Published var error: String?

    @Published var email = ""
    @Published var password = ""
    @Published var identifier = Constants.identifier
    @Published var token = Constants.token
    @Published var displayName = Constants.displayName

    func getCommunicationUserToken() -> CommunicationUserTokenModel? {
        isAuthenticating = true
        // MARK: modify below to get token from auth server.
        if !Constants.token.isEmpty && !Constants.identifier.isEmpty {
            let communicationUserTokenModel = CommunicationUserTokenModel(token: Constants.token, expiresOn: nil, communicationUserId: Constants.identifier)
            isAuthenticating = false
            return communicationUserTokenModel
        }

        isAuthenticating = false
        return nil
    }
}



This viewModel will initialize azure communication user token with constants that we defined earlier. You will have to modify this with your own token provider when applying to real project.

We will use getCommunicationUserToken function after implementing CallingViewModel shortly.

## ViewModels

### UIRepresentables

ACS renders stream with RenderView interface which conforms to UIView. In order to use this interface in SwiftUI, we have to use UIViewRepresentable.

Please visit here for detail about UIViewRepresentable.

Create VideoStreamView.swift file under Models folder and fill it with below content.

#### [VideoStreamView.swift](code/VideoStreamView.swift)

In [ ]:
import SwiftUI
import AzureCommunicationCalling

struct VideoStreamView: UIViewRepresentable {

    // This should be passed when you instantiate this class.
    // You should use value of `self.localRendererView` variable in your case.
    let view: RendererView

    func makeUIView(context: Context) -> UIView {
        return view // Fix#2: do NOT create a new view but rather use value returned from `renderer.createView()`
    }

    func updateUIView(_ uiView: UIView, context: Context) {}
}

We will use this struct to present our local video and remote participants’ videos.

### StreamViewModels
Create below files under Models folder.

#### [LocalVideoStreamModel.swift](code/LocalVideoStreamModel.swift)

In [ ]:
import SwiftUI
import AzureCommunicationCalling

class LocalVideoStreamModel: VideoStreamModel {
    public func createView(localVideoStream: LocalVideoStream?) {
        do {
            if let localVideoStream = localVideoStream {
                let renderer = try Renderer(localVideoStream: localVideoStream)
                self.renderer = renderer
                self.videoStreamView = VideoStreamView(view: (try renderer.createView()))
            }
        } catch {
            print("Failed starting VideoStreamView for \(String(describing: displayName)) : \(error.localizedDescription)")
        }
    }
}

#### [RemoteVideoStreamModel.swift](code/RemoteVideoStreamModel.swift)

In [ ]:
import SwiftUI
import AzureCommunicationCalling

class RemoteVideoStreamModel: VideoStreamModel, RemoteParticipantDelegate {
    @Published var isRemoteVideoStreamEnabled:Bool = false
    @Published var isMicrophoneMuted:Bool = false
    @Published var isSpeaking:Bool = false
    @Published var scalingMode: ScalingMode = .crop

    public var remoteParticipant: RemoteParticipant?

    public init(identifier: String, displayName: String, remoteParticipant: RemoteParticipant?) {
        self.remoteParticipant = remoteParticipant
        super.init(identifier: identifier, displayName: displayName)
        self.remoteParticipant!.delegate = self
        self.isMicrophoneMuted = false
        self.isSpeaking = false
    }

    func checkStream() {
        if let remoteParticipant = self.remoteParticipant,
           let videoStreams = remoteParticipant.videoStreams {
            if videoStreams.count > 0 && videoStreamView == nil {
                addStream(remoteVideoStream: videoStreams.first!)
            }
        }
    }

    private func addStream(remoteVideoStream: RemoteVideoStream) {
        do {
            let renderer = try Renderer(remoteVideoStream: remoteVideoStream)
            self.renderer = renderer
            self.videoStreamView = VideoStreamView(view: (try renderer.createView()))
            self.isRemoteVideoStreamEnabled = true
            print("Remote VideoStreamView started!")
        } catch {
            print("Failed starting VideoStreamView for \(String(describing: displayName)) : \(error.localizedDescription)")
        }
    }

    private func removeStream(stream: RemoteVideoStream?) {
        if stream != nil {
            self.renderer?.dispose()
            self.videoStreamView = nil
        }
    }

    func toggleScalingMode() {
        self.scalingMode = self.scalingMode == .crop ? ScalingMode.fit : ScalingMode.crop
        self.videoStreamView?.view.update(scalingMode: self.scalingMode)
    }

    func onParticipantStateChanged(_ remoteParticipant: RemoteParticipant!, args: PropertyChangedEventArgs!) {
        print("\n-------------------------")
        print("onParticipantStateChanged")
        print("-------------------------\n")

        if remoteParticipant.identity is CommunicationUserIdentifier {
            let remoteParticipantIdentity = remoteParticipant.identity as! CommunicationUserIdentifier
            print("RemoteParticipant identifier:  \(String(describing: remoteParticipantIdentity.identifier))")
            print("RemoteParticipant displayName \(String(describing: remoteParticipant.displayName))")
        } else {
            print("remoteParticipant.identity: UnknownIdentifier")
        }
    }

    func onIsMutedChanged(_ remoteParticipant: RemoteParticipant!, args: PropertyChangedEventArgs!) {
        print("\n----------------")
        print("onIsMutedChanged")
        print("----------------\n")
        self.isMicrophoneMuted = remoteParticipant.isMuted
        print("remoteParticipant.isMuted: \(remoteParticipant.isMuted)")
    }

    func onIsSpeakingChanged(_ remoteParticipant: RemoteParticipant!, args: PropertyChangedEventArgs!) {
        print("\n-------------------")
        print("onIsSpeakingChanged")
        print("-------------------\n")
        self.isSpeaking = remoteParticipant.isSpeaking
        print("remoteParticipant.isSpeaking: \(remoteParticipant.isSpeaking)")
    }

    func onDisplayNameChanged(_ remoteParticipant: RemoteParticipant!, args: PropertyChangedEventArgs!) {

    }

    func onVideoStreamsUpdated(_ remoteParticipant: RemoteParticipant!, args: RemoteVideoStreamsEventArgs!) {
        print("\n---------------------")
        print("onVideoStreamsUpdated")
        print("---------------------\n")

        if remoteParticipant.identity is CommunicationUserIdentifier {
            let remoteParticipantIdentity = remoteParticipant.identity as! CommunicationUserIdentifier
            print("RemoteParticipant identifier:  \(String(describing: remoteParticipantIdentity.identifier))")
            print("RemoteParticipant displayName \(String(describing: remoteParticipant.displayName))")

            if let addedStreams = args.addedRemoteVideoStreams {
                print("addedStreams: \(addedStreams.count)")
                if let stream = addedStreams.first {
                    self.addStream(remoteVideoStream: stream)
                }
            }

            if let removedStreams = args.removedRemoteVideoStreams {
                print("RemovedStreams: \(removedStreams.count)")
                self.removeStream(stream: removedStreams.first)
                self.isRemoteVideoStreamEnabled = false
            }
        }
    }
}

Above classes will be handling video streams and indicators for mic / video.

### Callkit manager
Create CallKitManager.swift under Models folder.

CallKit will give your app a native looking call experiences. Not only that there is one important reason that you have to use this CallKit that if you fail to report a call to CallKit after your app receives push notifications, the system will terminate your app. Read more [here](https://developer.apple.com/documentation/pushkit/pkpushregistrydelegate/2875784-pushregistry).

#### [CallKitManager.swift](code/CallKitManager.swift)

In [ ]:
import Foundation
import SwiftUI
import CallKit
import AzureCommunicationCalling
import AVFoundation

class CallKitManager: NSObject {
    private static var sharedInstance: CallKitManager?

    private let callController = CXCallController()
    private let provider: CXProvider


    static func shared() -> CallKitManager {
        if sharedInstance == nil {
            sharedInstance = CallKitManager()
        }
        return sharedInstance!
    }

    override init() {
        let configuration = CXProviderConfiguration()
        configuration.maximumCallGroups = 1
        configuration.maximumCallsPerCallGroup = 1
        configuration.supportsVideo = true
        configuration.iconTemplateImageData = UIImage(systemName: "video")?.pngData()
        provider = CXProvider(configuration: configuration)
        super.init()
        provider.setDelegate(self, queue: nil)
        print("callkitmanager init")
    }

    deinit {
        provider.invalidate()
    }

    // Start an outging call
    func startOutgoingCall(call: Call, callerDisplayName: String) {
        let callId = UUID(uuidString: call.callId)
        let handle = CXHandle(type: .generic, value: callerDisplayName)
        let startCallAction = CXStartCallAction(call: callId!, handle: handle)
        let transaction = CXTransaction(action: startCallAction)

        callController.request(transaction) { error in
            if let error = error {
                print("Error requesting CXStartCallAction transaction: \(error)")
            } else {
                print("Requested CXStartCallAction transaction successfully")
            }
        }

        provider.reportOutgoingCall(with: callId!, connectedAt: nil)
    }

    // End the call from the app. This is not needed when user end the call from the native CallKit UI
    func endCallFromLocal(callId: UUID, completion: @escaping (Bool) -> Void) {
        let endCallAction = CXEndCallAction(call: callId)
        let transaction = CXTransaction(action: endCallAction)

        callController.request(transaction, completion: { error in
            if let error = error {
                print("Error requesting CXEndCallAction transaction: \(error.localizedDescription)\n")
                completion(false)
            } else {
                print("Requested CXEndCallAction transaction successfully.\n")
                completion(true)
            }
        })
    }

    // This is normally called after receiving a VoIP Push Notification to handle incoming call
    func reportNewIncomingCall(incomingCallPushNotification: IncomingCallPushNotification, completion: @escaping (Bool) -> Void) {
        let callId = incomingCallPushNotification.callId
        let displayName = String(incomingCallPushNotification.fromDisplayName ?? "unknown")
        let update = CXCallUpdate()
        update.remoteHandle = CXHandle(type: .generic, value: "Incoming call from \(displayName)")

        self.provider.reportNewIncomingCall(with: callId, update: update) { (error) in
            if error == nil {
                completion(true)
            } else {
                completion(false)
            }
        }
    }

    func testReport(completion: @escaping (Bool) -> Void) {
        let update = CXCallUpdate()
        update.remoteHandle = CXHandle(type: .emailAddress, value: "Test")
        self.provider.reportNewIncomingCall(with: UUID(), update: update) { (error) in
            if error == nil {
                completion(true)
            } else {
                completion(false)
            }
        }
    }

    // Mute or unmute from the app. This is to sync the CallKit UI with app UI
    func setMuted(for call: Call, isMuted: Bool) {
        let setMutedAction = CXSetMutedCallAction(call: UUID(uuidString: call.callId)!, muted: isMuted)
        let transaction = CXTransaction(action: setMutedAction)
        callController.request(transaction, completion: { error in
            if let error = error {
                print("Error requesting CXSetMutedCallAction transaction: \(error)")
            } else {
                print("Requested CXSetMutedCallAction transaction successfully")
            }
        })
    }

    // This is to resume call from the app. When the interrupting call is ended from Remote,
    // provider::perform::CXSetMutedCallAction will not be called automatically
    func setHeld(with call: Call, isOnHold: Bool) {
        let setHeldCallAction = CXSetHeldCallAction(call: UUID(uuidString: call.callId)!, onHold: isOnHold)
        let transaction = CXTransaction(action: setHeldCallAction)
        callController.request(transaction, completion: { error in
            if let error = error {
                print("Error requesting CXSetHeldCallAction transaction: \(error)")
            } else {
                print("Requested CXSetHeldCallAction \(isOnHold) transaction successfully")
            }
        })
    }

    // Use this to notify CallKit the call is disconnected
    func reportCallEndedFromRemote(callId: UUID, reason: CXCallEndedReason) {
        print("reportCallEndedFromRemote.\n")
        provider.reportCall(with: callId, endedAt: Date(), reason: reason)
    }
}

#### [Extension.swift](code/Extension.swift)

In [ ]:
extension CallKitManager: CXProviderDelegate {
    func providerDidReset(_ provider: CXProvider) {
        print("providerDidReset")
    }

    func providerDidBegin(_: CXProvider) {}

    func provider(_: CXProvider, perform action: CXStartCallAction) {
        action.fulfill()
    }

    func provider(_: CXProvider, perform action: CXAnswerCallAction) {
        action.fulfill()
    }

    func provider(_: CXProvider, perform action: CXEndCallAction) {
        action.fulfill()
    }

    func provider(_: CXProvider, perform action: CXSetHeldCallAction) {
        action.fulfill()
    }

    func provider(_: CXProvider, perform action: CXSetMutedCallAction) {
        action.fulfill()
    }

    func provider(_: CXProvider, timedOutPerforming _: CXAction) {}

    func provider(_: CXProvider, didActivate _: AVAudioSession) {}

    func provider(_: CXProvider, didDeactivate _: AVAudioSession) {}
}

### Configrue NotificationHubs

Create NotificationViewModel.swfit under ViewModels folder.

#### [NotificationViewModel.swift](code/NotificationViewModel.swift)

In [ ]:
import Combine
import UserNotifications
import WindowsAzureMessaging

class NotificationViewModel: NSObject, ObservableObject, UNUserNotificationCenterDelegate, MSNotificationHubDelegate, MSInstallationLifecycleDelegate {
    private var notificationPresentationCompletionHandler: Any?
    private var notificationResponseCompletionHandler: Any?

    @Published var installationId: String = MSNotificationHub.getInstallationId()
    @Published var pushChannel: String = MSNotificationHub.getPushChannel()
    @Published var items = [MSNotificationHubMessage]()
    @Published var tags = MSNotificationHub.getTags()
    @Published var userId = MSNotificationHub.getUserId()

    let messageReceived = NotificationCenter.default
                .publisher(for: NSNotification.Name("MessageReceived"))

    let messageTapped = NotificationCenter.default
                .publisher(for: NSNotification.Name("MessageTapped"))

    func connectToHub() {
        let hubName = Constants.hubName
        let connectionString = Constants.connectionString

        if (!connectionString.isEmpty && !hubName.isEmpty)
        {
            UNUserNotificationCenter.current().delegate = self;
            MSNotificationHub.setLifecycleDelegate(self)
            MSNotificationHub.setDelegate(self)
            MSNotificationHub.start(connectionString: connectionString, hubName: hubName)

            print("connected to notification hub")
            addTags()
        }
    }

    func setUserId() {
        MSNotificationHub.setUserId(self.userId)
    }

    func addTags() {
        // Get language and country code for common tag values
        let language = Bundle.main.preferredLocalizations.first ?? "<undefined>"
        let countryCode = NSLocale.current.regionCode ?? "<undefined>"

        // Create tags with type_value format
        let languageTag = "language_" + language
        let countryCodeTag = "country_" + countryCode

        MSNotificationHub.addTags([languageTag, countryCodeTag])
    }

    func addTag(tag: String) {
        MSNotificationHub.addTag(tag)
    }

    @available(iOS 10.0, *)
    func userNotificationCenter(_ center: UNUserNotificationCenter, willPresent notification: UNNotification, withCompletionHandler completionHandler: @escaping (UNNotificationPresentationOptions) -> Void) {
        self.notificationPresentationCompletionHandler = completionHandler;
    }

    @available(iOS 10.0, *)
    func userNotificationCenter(_ center: UNUserNotificationCenter, didReceive response: UNNotificationResponse, withCompletionHandler completionHandler: @escaping () -> Void) {
        self.notificationResponseCompletionHandler = completionHandler;
    }

    func notificationHub(_ notificationHub: MSNotificationHub, didSave installation: MSInstallation) {
        DispatchQueue.main.async {
            self.installationId = installation.installationId
            self.pushChannel = installation.pushChannel
            print("notificationHub installation was successful.")
        }
    }

    func notificationHub(_ notificationHub: MSNotificationHub!, didFailToSave installation: MSInstallation!, withError error: Error!) {
        print("notificationHub installation failed.")
    }

    func notificationHub(_ notificationHub: MSNotificationHub, didReceivePushNotification message: MSNotificationHubMessage) {

        let userInfo = ["message": message]

        // Append receivedPushNotification message to self.items
        self.items.append(message)

        if (notificationResponseCompletionHandler != nil) {
            NSLog("Tapped Notification")
            NotificationCenter.default.post(name: NSNotification.Name("MessageTapped"), object: nil, userInfo: userInfo)
        } else {
            NSLog("Notification received in the foreground")
            NotificationCenter.default.post(name: NSNotification.Name("MessageReceived"), object: nil, userInfo: userInfo)
        }

        // Call notification completion handlers.
        if (notificationResponseCompletionHandler != nil) {
            (notificationResponseCompletionHandler as! () -> Void)()
            notificationResponseCompletionHandler = nil
        }
        if (notificationPresentationCompletionHandler != nil) {
            (notificationPresentationCompletionHandler as! (UNNotificationPresentationOptions) -> Void)([])
            notificationPresentationCompletionHandler = nil
        }
    }
}

When *connectToHub()* function is called, your app will be connected to Azure Notification Hub and ready to receive standard push notifications. But in order to receive notifications for Calls, you will have to register voip type deviceToken for ACS separately. We'll cover this after we configure ACS connection in next section.

Press *Command + Option + P* and note that tab bar navigation is applied as below.

### Connect to Azure Communication Services
Create CallingViewModel.swift under ViewModels folder.

This class handles all events related to receiving and starting calls. On init(), we initialize PKPushRegistry with voIP pushType and sets delegate to it self. This enables to receive push notifications in PKPushRegistryDelegate. Once we get push notifications, we have to report incoming call to CallKitManger.reportNewIncomingCall() that we have created earlier.

#### [CallingViewModel.swift](code/CallingViewModel.swift)

In [ ]:
import Combine
import PushKit
import CallKit
import AVFoundation
import AzureCommunicationCalling

class CallingViewModel: NSObject, ObservableObject {
    private static var sharedInstance: CallingViewModel?
    private(set) var callClient: CallClient?
    private(set) var callAgent: CallAgent?
    private(set) var call: Call?
    private(set) var deviceManager: DeviceManager?
    private(set) var localVideoStream: LocalVideoStream?
    private var pushRegistry: PKPushRegistry
    private var voIPToken: Data?

    @Published var hasCallAgent: Bool = false
    @Published var callState: CallState = CallState.none
    @Published var localVideoStreamModel: LocalVideoStreamModel?
    @Published var remoteVideoStreamModels: [RemoteVideoStreamModel] = []
    @Published var isLocalVideoStreamEnabled:Bool = false
    @Published var isMicrophoneMuted:Bool = false
    @Published var incomingCallPushNotification: IncomingCallPushNotification?
    @Published var callee: String = Constants.callee
    @Published var groupId: String = "29228d3e-040e-4656-a70e-890ab4e173e5"

    static func shared() -> CallingViewModel {
        if sharedInstance == nil {
            sharedInstance = CallingViewModel()

            // This is to initialize CallKit properly before requesting first outgoing/incoming call
            _ = CallKitManager.shared()
        }
        return sharedInstance!
    }

    override init() {
        callClient = CallClient()
        pushRegistry = PKPushRegistry(queue: DispatchQueue.main)
        super.init()
        pushRegistry.delegate = self
        pushRegistry.desiredPushTypes = [PKPushType.voIP]
    }

    func initCallAgent(communicationUserTokenModel: CommunicationUserTokenModel, displayName: String?, completion: @escaping (Bool) -> Void) {
        if let communicationUserId = communicationUserTokenModel.communicationUserId,
           let token = communicationUserTokenModel.token {
            do {
                let communicationTokenCredential = try CommunicationTokenCredential(token: token)
                let callAgentOptions = CallAgentOptions()
                callAgentOptions?.displayName = displayName ?? communicationUserId
                self.callClient?.createCallAgent(userCredential: communicationTokenCredential, options: callAgentOptions) { (callAgent, error) in
                    if self.callAgent != nil {
                        self.callAgent?.delegate = nil
                    }
                    self.callAgent = callAgent
                    self.callAgent?.delegate = self
                    self.hasCallAgent = true

                    print("CallAgent successfully created.\n")
                    completion(true)
                }
            } catch {
                print("Error: \(error.localizedDescription)")
                completion(false)
            }
        } else {
            print("Invalid communicationUserTokenModel.\n")
        }
    }

    func initPushNotification() {
        self.callAgent?.registerPushNotifications(deviceToken: self.voIPToken, completionHandler: { (error) in
            if(error == nil) {
                print("Successfully registered to VoIP push notification.\n")
            } else {
                print("Failed to register VoIP push notification.\(String(describing: error))\n")
            }
        })
    }

    func getCall(callId: UUID) -> Call? {
        if let call = self.call {
            print("incoming callId: \(call.callId.uppercased())")
            print("push callId: \(callId)")

            if let currentCallId = UUID(uuidString: call.callId) {
                if currentCallId == callId {
                    return call
                } else {
                    return nil
                }
            } else {
                print("Error parsing callId from currentCall.\n")
            }
        } else {
            print("call not exist in CallingViewModel!!!.\n")
        }
        return nil
    }

    func resetCallAgent() {
        if let callAgent = self.callAgent {
            unRegisterVoIP()
            callAgent.delegate = nil
            self.callAgent = nil
        } else {
            print("callAgent not found.\n")
        }
        self.hasCallAgent = false
    }

    func getDeviceManager(completion: @escaping (Bool) -> Void) {
        requestVideoPermission { success in
            if success {
                self.callClient?.getDeviceManager(completionHandler: { (deviceManager, error) in
                    if (error == nil) {
                        print("Got device manager instance")
                        self.deviceManager = deviceManager

                        if let videoDeviceInfo: VideoDeviceInfo = deviceManager?.getCameraList()?.first {
                            self.localVideoStream = LocalVideoStream(camera: videoDeviceInfo)
                            self.localVideoStreamModel = LocalVideoStreamModel(identifier: Constants.identifier, displayName: Constants.displayName)
                            print("LocalVideoStream instance initialized.")
                            completion(true)
                        } else {
                            print("LocalVideoStream instance initialize faile.")
                            completion(false)
                        }
                    } else {
                        print("Failed to get device manager instance: \(String(describing: error))")
                        completion(false)
                    }
                })
            } else {
                print("Permission denied.\n")
                completion(false)
            }
        }
    }

    // MARK: Request RecordPermission
    func requestRecordPermission(completion: @escaping (Bool) -> Void) {
        let audioSession = AVAudioSession.sharedInstance()
        switch audioSession.recordPermission {
        case .undetermined:
            audioSession.requestRecordPermission { granted in
                if granted {
                    completion(true)
                } else {
                    print("User did not grant audio permission")
                    completion(false)
                }
            }
        case .denied:
            print("User did not grant audio permission, it should redirect to Settings")
            completion(false)
        case .granted:
            completion(true)
        @unknown default:
            print("Audio session record permission unknown case detected")
            completion(false)
        }
    }

    // MARK: Request VideoPermission
    func requestVideoPermission(completion: @escaping (Bool) -> Void) {
        switch AVCaptureDevice.authorizationStatus(for: .video) {
        case .notDetermined:
            AVCaptureDevice.requestAccess(for: .video) { authorized in
                if authorized {
                    completion(true)
                } else {
                    print("User did not grant video permission")
                    completion(false)
                }
            }
        case .restricted, .denied:
            print("User did not grant video permission, it should redirect to Settings")
            completion(false)
        case .authorized:
            completion(true)
        @unknown default:
            print("AVCaptureDevice authorizationStatus unknown case detected")
            completion(false)
        }
    }

    // MARK: Configure AudioSession
    func configureAudioSession() {
        let audioSession = AVAudioSession.sharedInstance()
        do {
            if audioSession.category != .playAndRecord {
                try audioSession.setCategory(AVAudioSession.Category.playAndRecord,
                                             options: AVAudioSession.CategoryOptions.allowBluetooth)
                try audioSession.setActive(true, options: .notifyOthersOnDeactivation)
            }
            if audioSession.mode != .voiceChat {
                try audioSession.setMode(.voiceChat)
            }
        } catch {
            print("Error configuring AVAudioSession: \(error.localizedDescription)")
        }
    }

    func startVideo(call: Call, localVideoStream: LocalVideoStream) -> Void {
        requestVideoPermission { success in
            if success {
                if let localVideoStreamModel = self.localVideoStreamModel {
                    call.startVideo(stream: localVideoStream) { error in
                        if error != nil {
                            print("LocalVideo failed to start.\n")
                        } else {
                            print("LocalVideo started successfully.\n")
                            localVideoStreamModel.createView(localVideoStream: localVideoStream)
                            self.isLocalVideoStreamEnabled = true
                        }
                    }
                }
            } else {
                print("Permission denied.\n")
            }
        }
    }

    func stopVideo(competion: @escaping (Bool) -> Void) {
        if let call = self.call {
            call.stopVideo(stream: self.localVideoStream) { error in
                if error != nil {
                    print("LocalVideo failed to stop.\n")
                    competion(false)
                } else {
                    print("LocalVideo stopped successfully.\n")
                    if self.localVideoStreamModel != nil {
                        self.isLocalVideoStreamEnabled = false
                    }
                    competion(true)
                }
            }
        }
    }

    func stopVideo() {
        self.stopVideo { success in
            if success {
                self.localVideoStreamModel?.renderer?.dispose()
                self.localVideoStreamModel?.renderer = nil
                self.localVideoStreamModel?.videoStreamView = nil
            }
        }
    }

    func joinGroup() {
        requestRecordPermission { success in
            guard success else {
                print("recordPermission not authorized.")
                return
            }

            if let callAgent = self.callAgent {
                let groupCallLocator = GroupCallLocator(groupId: UUID(uuidString: self.groupId))
                let joinCallOptions = JoinCallOptions()

                self.getDeviceManager { _ in
                    if let localVideoStream = self.localVideoStream {
                        let videoOptions = VideoOptions(localVideoStream: localVideoStream)

                        joinCallOptions?.videoOptions = videoOptions

                        self.call = callAgent.join(with: groupCallLocator, joinCallOptions: joinCallOptions)

                        self.call?.delegate = self
                        self.startVideo(call: self.call!, localVideoStream: localVideoStream)
                        CallKitManager.shared().startOutgoingCall(call: self.call!, callerDisplayName: Constants.displayName)
                        print("outgoing call started.")
                    } else {
                        self.call = self.callAgent?.join(with: groupCallLocator, joinCallOptions: joinCallOptions)
                        CallKitManager.shared().startOutgoingCall(call: self.call!, callerDisplayName: Constants.displayName)
                        self.call?.delegate = self
                        print("outgoing call started.")
                    }
                }
            } else {
                print("callAgent not initialized.\n")
            }
        }
    }

    func startCall() {
        requestRecordPermission { success in
            guard success else {
                print("recordPermission not authorized.")
                return
            }

            if let callAgent = self.callAgent {
                let callees:[CommunicationUserIdentifier] = [CommunicationUserIdentifier(identifier: self.callee)]
                let startCallOptions = StartCallOptions()

                self.getDeviceManager { _ in
                    if let localVideoStream = self.localVideoStream {
                        let videoOptions = VideoOptions(localVideoStream: localVideoStream)
                        startCallOptions?.videoOptions = videoOptions

                        self.call = callAgent.call(participants: callees, options: startCallOptions)
                        self.call?.delegate = self
                        self.startVideo(call: self.call!, localVideoStream: localVideoStream)
                        CallKitManager.shared().startOutgoingCall(call: self.call!, callerDisplayName: Constants.displayName)
                        print("outgoing call started.")
                    } else {
                        self.call = callAgent.call(participants: callees, options: startCallOptions)
                        CallKitManager.shared().startOutgoingCall(call: self.call!, callerDisplayName: Constants.displayName)
                        self.call?.delegate = self
                        print("outgoing call started.")
                    }
                }
            } else {
                print("callAgent not initialized.\n")
            }
        }
    }

    // Accept incoming call
    func acceptCall(callId: UUID, completion: @escaping (Bool) -> Void) {
        if self.incomingCallPushNotification == nil {
            self.requestRecordPermission { authorized in
                if authorized {
                    if let call = self.getCall(callId: callId) {
                        let acceptCallOptions = AcceptCallOptions()

                        self.getDeviceManager { _ in
                            if let localVideoStream = self.localVideoStream {
                                let videoOptions = VideoOptions(localVideoStream: localVideoStream)
                                acceptCallOptions?.videoOptions = videoOptions
                                // MARK: startVideo when connection has made
                                self.startVideo(call: call, localVideoStream: localVideoStream)
                            }

                            call.accept(options: acceptCallOptions) { error in
                                if let error = error {
                                    print("Failed to accpet incoming call: \(error.localizedDescription)\n")
                                    completion(false)
                                } else {
                                    print("Incoming call accepted with acceptCallOptions.\n")
                                    completion(true)
                                }
                            }
                        }
                    } else {
                        print("Call not found when trying to accept.\n")
                        completion(false)
                    }
                } else {
                    print("recordPermission not authorized.")
                }
            }
        } else {
            print("incomingCallPushNotification not processed yet")
            DispatchQueue.main.asyncAfter(deadline: DispatchTime.now() + 2) {
                self.incomingCallPushNotification = nil
                self.acceptCall(callId: callId) { _ in }
            }
        }
    }

    func endCall() -> Void {
        print("endCall requested from App.\n")
        if let call = self.call {
            call.hangup(options: HangupOptions()) { error in
                if let error = error {
                    print("hangup failed: \(error.localizedDescription).\n")
                } else {
                    print("hangup succeed.\n")
                }
            }
        } else {
            print("Call not found.\n")
        }
    }

    func endCall(callId: UUID, completion: @escaping (Bool) -> Void) {
        print("endCall requested from CallKit.\n")
        if let call = self.getCall(callId: callId) {
            call.hangup(options: HangupOptions()) { error in
                if let error = error {
                    print("hangup failed: \(error.localizedDescription).\n")
                    completion(false)
                } else {
                    print("hangup succeed.\n")
                    completion(true)
                }
            }
        } else {
            print("Call not found when trying to hangup.\n")
            completion(false)
        }
    }

    func mute() {
        if let call = self.call {
            if call.isMicrophoneMuted {
                call.unmute(completionHandler:{ (error) in
                    if error == nil {
                        print("Successfully un-muted")
                        self.isMicrophoneMuted = false
                    } else {
                        print("Failed to unmute")
                    }
                })
            } else {
                call.mute(completionHandler: { (error) in
                    if error == nil {
                        print("Successfully muted")
                        self.isMicrophoneMuted = true
                    } else {
                        print("Failed to mute")
                    }
                })
            }
        }
    }

    func toggleVideo() {
        if let call = self.call,
           let localVideoStream = self.localVideoStream {
            if isLocalVideoStreamEnabled {
                stopVideo()
            } else {
                startVideo(call: call, localVideoStream: localVideoStream)
            }
        }
    }
}

Add ACS and PushKit delegates into CallingViewModel as below.

#### [CallAgentDelegate.swift](code/CallAgentDelegate.swift)

In [ ]:
extension CallingViewModel: CallAgentDelegate {
    func onCallsUpdated(_ callAgent: CallAgent!, args: CallsUpdatedEventArgs!) {
        print("\n---------------")
        print("onCallsUpdated")
        print("---------------\n")

        if let addedCall = args.addedCalls?.first(where: {$0.isIncoming }) {
            print("addedCalls: \(args.addedCalls.count)")
            self.call = addedCall
            self.call?.delegate = self
            self.callState = addedCall.state
            self.isMicrophoneMuted = addedCall.isMicrophoneMuted
        }

        if let removedCall = args.removedCalls?.first {
            print("removedCalls: \(args.removedCalls.count)\n")
            let removedCallUUID = UUID(uuidString: removedCall.callId)
            // MARK: report CallKitManager for endCall.
            CallKitManager.shared().reportCallEndedFromRemote(callId: removedCallUUID!, reason: CXCallEndedReason.remoteEnded)

            if let call = self.call {
                print("call removed.\n")
                if call.callId == removedCall.callId {
                    self.callState = removedCall.state
                    self.call?.delegate = nil
                    self.call = nil
                }
            } else {
                print("\ncall removed before initizliaztion.\n")
            }
        } else {
            print("removedCall: \(String(describing: args.removedCalls))")
            if let incomingCallPushNotification = self.incomingCallPushNotification {
                CallKitManager.shared().reportCallEndedFromRemote(callId: incomingCallPushNotification.callId, reason: CXCallEndedReason.remoteEnded)
            }
        }
    }
}

#### [CallDelegate.swift](code/CallDelegate.swift)

In [ ]:
extension CallingViewModel: CallDelegate {
    func onCallStateChanged(_ call: Call!, args: PropertyChangedEventArgs!) {
        print("\n----------------------------------")
        print("onCallStateChanged: \(String(reflecting: call.state))")
        print("----------------------------------\n")
        self.callState = call.state

        if call.state == .connected {
            print("call state connected: \(String(reflecting: call.state))")

            //if let localVideoStream = self.localVideoStream { self.startVideo(call: call, localVideoStream: localVideoStream) }
        }

        if call.state == .disconnected || call.state == .none {
            self.stopVideo()
            self.remoteVideoStreamModels.forEach({ (remoteVideoStreamModel) in
                remoteVideoStreamModel.renderer?.dispose()
                remoteVideoStreamModel.videoStreamView = nil
                remoteVideoStreamModel.remoteParticipant?.delegate = nil
            })
            self.remoteVideoStreamModels = []
        }
    }

    func onRemoteParticipantsUpdated(_ call: Call!, args: ParticipantsUpdatedEventArgs!) {
        print("\n---------------------------")
        print("onRemoteParticipantsUpdated")
        print("---------------------------\n")

        if let addedParticipants = args.addedParticipants {
            if addedParticipants.count > 0 {
                print("addedParticipants: \(String(describing: args.addedParticipants.count))")

                addedParticipants.forEach { (remoteParticipant) in
                    if remoteParticipant.identity is CommunicationUserIdentifier {
                        let communicationUserIdentifier = remoteParticipant.identity as! CommunicationUserIdentifier
                        print("addedParticipant identifier:  \(String(describing: communicationUserIdentifier))")
                        print("addedParticipant displayName \(String(describing: remoteParticipant.displayName))")
                        print("addedParticipant streams \(String(describing: remoteParticipant.videoStreams.count))")

                        let remoteVideoStreamModel = RemoteVideoStreamModel(identifier: communicationUserIdentifier.identifier, displayName: remoteParticipant.displayName, remoteParticipant: remoteParticipant)
                        remoteVideoStreamModels.append(remoteVideoStreamModel)
                    }
                }
            }
        }

        if let removedParticipants = args.removedParticipants {
            if removedParticipants.count > 0 {
                print("removedParticipants: \(String(describing: args.removedParticipants.count))")

                removedParticipants.forEach { (remoteParticipant) in
                    if remoteParticipant.identity is CommunicationUserIdentifier {
                        let communicationUserIdentifier = remoteParticipant.identity as! CommunicationUserIdentifier
                        print("removedParticipant identifier:  \(String(describing: communicationUserIdentifier))")
                        print("removedParticipant displayName \(String(describing: remoteParticipant.displayName))")

                        if let removedIndex = remoteVideoStreamModels.firstIndex(where: {$0.identifier == communicationUserIdentifier.identifier}) {
                            let remoteVideoStreamModel = remoteVideoStreamModels[removedIndex]
                            remoteVideoStreamModel.remoteParticipant?.delegate = nil
                            remoteVideoStreamModel.renderer?.dispose()
                            remoteVideoStreamModel.videoStreamView = nil
                            remoteVideoStreamModels.remove(at: removedIndex)
                        }
                    }
                }
            }
        }
    }

    func onLocalVideoStreamsChanged(_ call: Call!, args: LocalVideoStreamsUpdatedEventArgs!) {
        print("\n--------------------------")
        print("onLocalVideoStreamsChanged")
        print("--------------------------\n")

        if let addedStreams = args.addedStreams {
            print("addedStreams: \(addedStreams.count)")
        }

        if let removedStreams = args.removedStreams {
            print("removedStreams: \(removedStreams.count)")
        }
    }
}

#### [PKPushRegistryDelegate.swift](code/PKPushRegistryDelegate.swift)

In [ ]:
extension CallingViewModel: PKPushRegistryDelegate {
    func unRegisterVoIP() {
        self.callAgent?.unRegisterPushNotifications(completionHandler: { (error) in
            if (error != nil) {
                print("Register of push notification failed, please try again.\n")
            } else {
                print("Unregister of push notification was successful.\n")
            }
        })
    }

    func pushRegistry(_ registry: PKPushRegistry, didUpdate pushCredentials: PKPushCredentials, for type: PKPushType) {
        let deviceToken = pushCredentials.token.map { String(format: "%02x", $0) }.joined()
                print("pushRegistry -> deviceToken :\(deviceToken)")

        self.voIPToken = pushCredentials.token
    }

    func pushRegistry(_ registry: PKPushRegistry, didInvalidatePushTokenFor type: PKPushType) {
        print("pusRegistry invalidated.")
    }

    func pushRegistry(_ registry: PKPushRegistry, didReceiveIncomingPushWith payload: PKPushPayload, for type: PKPushType, completion: @escaping () -> Void) {
        let dictionaryPayload = payload.dictionaryPayload
        print("dictionaryPayload: \(dictionaryPayload)")

        if type == .voIP {
            if let incomingCallPushNotification = IncomingCallPushNotification.fromDictionary(payload.dictionaryPayload) {
                self.configureAudioSession()
                CallKitManager.shared().reportNewIncomingCall(incomingCallPushNotification: incomingCallPushNotification) { success in
                    if success {
                        print("Handling of report incoming call was succesful.\n")
                        completion()
                    } else {
                        print("Handling of report incoming call failed.\n")
                        completion()
                    }
                }

                if self.callAgent == nil {
                    self.incomingCallPushNotification = incomingCallPushNotification

                    print("CallAgent not found.\nConnecting to Communication Services...\n")
                    // MARK: generate communicationUserToken from stored data.
                    let token = Constants.token
                    let identifier = Constants.identifier
                    let displayName = Constants.displayName

                    if !token.isEmpty && !identifier.isEmpty {
                        let communicationUserToken = CommunicationUserTokenModel(token: token, expiresOn: nil, communicationUserId: identifier)
                        self.initCallAgent(communicationUserTokenModel: communicationUserToken, displayName: displayName) { (success) in
                            if success {
                                self.initPushNotification()

                                self.callAgent?.handlePush(notification: incomingCallPushNotification, completionHandler: { error in
                                    if (error != nil) {
                                        print("Handling of push notification to call failed: \(error.debugDescription)\n")
                                    } else {
                                        print("Handling of push notification to call was successful.\n")
                                        self.incomingCallPushNotification = nil
                                    }
                                })
                            } else {
                                print("initCallAgent failed.\n")
                            }
                        }
                    } else {
                        // MARK: no token found, unregister push notification when signing out.
                        print("No token found,\n")
                    }
                }
            }
        } else {
            print("Pushnotification is not type of voIP.\n")
        }
    }
}

Register push notification
Now we are done with viewModels and ready to get some call notifications.

Add checkToken() function in ContentView.swift as below and initialize ACS callAgent class. Once callAgent is initialized, connect to notification hub using connectToHub() function in NotificationViewModel.swfit. With MSNotificationHubDelegate, we will receive didSave delegate event from notification hub and we are ready to register our voip token to ACS.

Change didSave function in **NotificationViewModel.swift** as below.

#### [NotificationViewModel_edit.swift](code/NotificationViewModel_edit.swift)

In [ ]:
func notificationHub(_ notificationHub: MSNotificationHub, didSave installation: MSInstallation) {
      DispatchQueue.main.async {
          self.installationId = installation.installationId
          self.pushChannel = installation.pushChannel
          print("notificationHub installation was successful.")
          CallingViewModel.shared().initPushNotification()
      }
  }

Look for *initPushNotification()* function in **CallingViewModel.swift** and find that this func is finally accessing self.voIPToken that was stored from *didUpdate* delegate of *pushRegistry* by calling self.callAgent?.registerPushNotifications(deviceToken: self.voIPToken, completionHandler: ((Error?) -> Void)!)

Well done!! we are ready to get voip call notifications now.

Note that didUpdate delegate of pushRegistry stores updated deviceToken to self.voIPToken which we will use with ACS callAgent for pushNotification register in next section.

## Views

Since we are done with viewModels, let’s tackle some views to actually make and receive a call.

But before we start drawing our views, we have to initialize our viewModels first. Open AzureCommunicationVideoCallingSampleApp.swift and modify the content as below. We will initialize viewModels we have created earlier and set them as environmentObject so that our child views can access them.

#### [AzureCommunicationVideoCallingSampleApp_edit.swift](code/AzureCommunicationVideoCallingSampleApp_edit.swift)

In [ ]:
import SwiftUI

@main
struct AzureCommunicationVideoCallingSampleApp: App {
    @Environment(\.scenePhase) private var scenePhase
    @StateObject private var authenticationViewModel = AuthenticationViewModel()
    @StateObject private var notificationViewModel = NotificationViewModel()
    @StateObject private var callingViewModel = CallingViewModel.shared()

    init() {
        // Fill in DevSettings.plist for AzureNotificationHubs hubName and connectionString.
        Constants.hubName = getPlistInfo(resourceName: "DevSettings", key: "HUB_NAME")
        Constants.connectionString = getPlistInfo(resourceName: "DevSettings", key: "CONNECTION_STRING")

        // Fill in FirstUser.plist with displayName, identifier, token and receiver identifier to test call feature.
        // Change resouceName to "FirstUser" or "SecondUser" to deploy different credentials.
        let resourceName = "FirstUser"
        Constants.displayName = getPlistInfo(resourceName: resourceName, key: "DISPLAYNAME")
        Constants.identifier = getPlistInfo(resourceName: resourceName, key: "IDENTIFIER")
        Constants.token = getPlistInfo(resourceName: resourceName, key: "TOKEN")
        Constants.callee = getPlistInfo(resourceName: resourceName, key: "CALLEE")
    }

    var body: some Scene {
        WindowGroup {
            ContentView()
                .environmentObject(authenticationViewModel)
                .environmentObject(notificationViewModel)
                .environmentObject(callingViewModel)
        }
        .onChange(of: scenePhase) { (newScenePhase) in
            switch newScenePhase {
            case .active:
                print("scene is now active!")
            case .inactive:
                print("scene is now inactive!")
            case .background:
                print("scene is now in the background!")
            @unknown default:
                print("Apple must have added something new!")
            }
        }
    }
}

Because this sample also covers group call, we will create Grid layout view that will automatically resize each participant’s view size.

Create Grid.swift and GridLayout.swift([adopted from Stanford cs193p GridLayout.swift](https://cs193p.sites.stanford.edu/))

#### [Grid.swift](code/Grid.swift)

In [ ]:
import SwiftUI

struct Grid<Item, ItemView>: View where Item: Identifiable, ItemView: View {
    var items: [Item]
    var viewForItem: (Item) -> ItemView

    init(_ items: [Item], _ viewForItem: @escaping (Item) -> ItemView) {
        self.items = items
        self.viewForItem = viewForItem
    }
    var body: some View {
        GeometryReader { geometry in
            self.body(for: GridLayout(itemCount: self.items.count, in: geometry.size))
        }
    }

    func body(for layout: GridLayout) -> some View {
        ForEach(items) { item in
            self.body(for: item, in: layout)
        }
    }

    func body(for item: Item, in layout: GridLayout) -> some View {
        let index = self.index(of: item)
        return viewForItem(item)
            .frame(width: layout.itemSize.width, height: layout.itemSize.height)
            .position(layout.location(ofItemAt: index))

    }

    func index(of item: Item) -> Int {
        for index in 0..<items.count {
            if items[index].id == item.id {
                return index
            }
        }
        return 0 // TODO: bogus
    }
}

#### [GridLayout.swift](code/GridLayout.swift)

In [ ]:
import SwiftUI

struct GridLayout {
    var size: CGSize
    var rowCount: Int = 0
    var columnCount: Int = 0

    init(itemCount: Int, nearAspectRatio desiredAspectRatio: Double = 1, in size: CGSize) {
        self.size = size
        // if our size is zero width or height or the itemCount is not > 0
        // then we have no work to do (because our rowCount & columnCount will be zero)
        guard size.width != 0, size.height != 0, itemCount > 0 else { return }
        // find the bestLayout
        // i.e., one which results in cells whose aspectRatio
        // has the smallestVariance from desiredAspectRatio
        // not necessarily most optimal code to do this, but easy to follow (hopefully)
        var bestLayout: (rowCount: Int, columnCount: Int) = (1, itemCount)
        var smallestVariance: Double?
        let sizeAspectRatio = abs(Double(size.width/size.height))
        for rows in 1...itemCount {
            let columns = (itemCount / rows) + (itemCount % rows > 0 ? 1 : 0)
            if (rows - 1) * columns < itemCount {
                let itemAspectRatio = sizeAspectRatio * (Double(rows)/Double(columns))
                let variance = abs(itemAspectRatio - desiredAspectRatio)
                if smallestVariance == nil || variance < smallestVariance! {
                    smallestVariance = variance
                    bestLayout = (rowCount: rows, columnCount: columns)
                }
            }
        }
        rowCount = bestLayout.rowCount
        columnCount = bestLayout.columnCount
    }

    var itemSize: CGSize {
        if rowCount == 0 || columnCount == 0 {
            return CGSize.zero
        } else {
            return CGSize(
                width: size.width / CGFloat(columnCount),
                height: size.height / CGFloat(rowCount)
            )
        }
    }

    func location(ofItemAt index: Int) -> CGPoint {
        if rowCount == 0 || columnCount == 0 {
            return CGPoint.zero
        } else {
            return CGPoint(
                x: (CGFloat(index % columnCount) + 0.5) * itemSize.width,
                y: (CGFloat(index / columnCount) + 0.5) * itemSize.height
            )
        }
    }
}

### Stream Views

Create StreamView.swift for rendering video streams. This view will display remoteParticipants’s mic and camera status along with video stream and display name.

#### [StreamView.swift](code/StreamView.swift)

In [ ]:
import SwiftUI

struct StreamView: View {
    @StateObject var remoteVideoStreamModel: RemoteVideoStreamModel
    @State var isMicrophoneMuted:Bool = false
    @State var isSpeaking:Bool = false

    var body: some View {
        ZStack {
            if remoteVideoStreamModel.videoStreamView != nil {
                remoteVideoStreamModel.videoStreamView!
            } else {
                Rectangle()
                    .foregroundColor(.black)
                    .edgesIgnoringSafeArea(.all)
                Text("Initializing video...")
                    .foregroundColor(.white)
            }
            VStack {
                HStack {
                    Spacer()
                    Spacer()
                    Text(remoteVideoStreamModel.displayName)
                        .foregroundColor(.secondary)
                        .font(.subheadline)
                    Image(systemName: self.isMicrophoneMuted ? "speaker.slash" : "speaker.wave.2")
                        .foregroundColor(.secondary)
                        .font(.subheadline)
                        .padding()
                    Spacer()
                }
                .padding(.top, 30)
                Spacer()
            }
        }
        .onTapGesture(count: 2) {
            print("double tapped!")
            remoteVideoStreamModel.toggleScalingMode()
        }
        .edgesIgnoringSafeArea(.all)
        .onReceive(remoteVideoStreamModel.$isMicrophoneMuted, perform: { isMicrophoneMuted in
            self.isMicrophoneMuted = isMicrophoneMuted
            print("isMicrophoneMuted: \(isMicrophoneMuted)")
        })
        .onReceive(remoteVideoStreamModel.$isSpeaking, perform: { isSpeaking in
            self.isSpeaking = isSpeaking
            print("isSpeaking: \(isSpeaking)")
        })
        .onAppear {
            remoteVideoStreamModel.checkStream()
        }
    }
}


#### Call Views

#### [CallView.swift](code/CallView.swift)

In [ ]:
import SwiftUI

struct CallView: View {
    @EnvironmentObject var authenticationViewModel: AuthenticationViewModel
    @EnvironmentObject var callingViewModel: CallingViewModel

    var body: some View {
        VStack(alignment: .center, spacing: 0) {
            if callingViewModel.remoteVideoStreamModels.count == 0 {
                VStack {
                    Text("Waiting for other participants...")
                    Button(action: { callingViewModel.endCall() }, label: {
                       HStack {
                           Spacer()
                           Text("Leave Group Call")
                           Spacer()
                       }
                   })

                }
            } else if callingViewModel.remoteVideoStreamModels.count == 1 {
                DirectCallView()
                    .environmentObject(callingViewModel)
            } else {
                GroupCallView()
                    .environmentObject(callingViewModel)
            }
        }
    }
}

struct CallView_Previews: PreviewProvider {
    static var previews: some View {
        CallView()
            .environmentObject(AuthenticationViewModel())
            .environmentObject(CallingViewModel())
    }
}

#### [DirectCallView.swift](code/DirectCallView.swift)

In [ ]:
import SwiftUI

struct DirectCallView: View {
    @EnvironmentObject var authenticationViewModel: AuthenticationViewModel
    @EnvironmentObject var callingViewModel: CallingViewModel

    private var selectedAnchor: Alignment = .topLeading

    var body: some View {
        ZStack {
            if !callingViewModel.remoteVideoStreamModels.isEmpty {
                StreamView(remoteVideoStreamModel: callingViewModel.remoteVideoStreamModels.first!)
            } else {
                Rectangle()
                    .edgesIgnoringSafeArea(.all)
            }
            VStack {
                GeometryReader { geometry in
                    if callingViewModel.localVideoStreamModel != nil {
                        callingViewModel.localVideoStreamModel?.videoStreamView
                            .cornerRadius(16)
                            .frame(width: geometry.size.width / 3, height: geometry.size.height / 3)
                            .padding([.top, .leading], 30)
                    } else {
                        Rectangle()
                            .foregroundColor(/*@START_MENU_TOKEN@*/.blue/*@END_MENU_TOKEN@*/)
                            .cornerRadius(16)
                            .frame(width: geometry.size.width / 3, height: geometry.size.height / 3)
                            .padding([.top, .leading], 30)
                    }
                }
                Spacer()
                HStack {
                    Button(action: { callingViewModel.toggleVideo() }, label: {
                        HStack {
                            Spacer()
                            if callingViewModel.isLocalVideoStreamEnabled {
                                Image(systemName: "video")
                                    .padding()
                            } else {
                                Image(systemName: "video.slash")
                                    .padding()
                            }
                            Spacer()
                        }
                    })
                    Button(action: { callingViewModel.mute() }, label: {
                        HStack {
                            Spacer()
                            if callingViewModel.isMicrophoneMuted {
                                Image(systemName: "speaker.slash")
                                    .padding()
                            } else {
                                Image(systemName: "speaker.wave.2")
                                    .padding()
                            }
                            Spacer()
                        }
                    })
                    Button(action: { callingViewModel.endCall() }, label: {
                        HStack {
                            Spacer()
                            Image(systemName: "phone.down")
                                .foregroundColor(.red)
                                .padding()
                            Spacer()
                        }
                    })
                }
            }
            .font(.largeTitle)
        }
    }
}

struct DirectCall_Previews: PreviewProvider {
    static var previews: some View {
        DirectCallView()
            .environmentObject(AuthenticationViewModel())
            .environmentObject(CallingViewModel())
    }
}

#### [GroupCallView.swift](code/GroupCallView.swift)

In [ ]:
import SwiftUI

struct GroupCallView: View {
    @EnvironmentObject var authenticationViewModel: AuthenticationViewModel
    @EnvironmentObject var callingViewModel: CallingViewModel

    var body: some View {
        ZStack {
            Grid(callingViewModel.remoteVideoStreamModels) { stream in
                StreamView(remoteVideoStreamModel: stream)
                    .padding()
            }

            VStack(alignment: .center) {
                Spacer()
                HStack {
                    Button(action: { callingViewModel.toggleVideo() }, label: {
                        HStack {
                            Spacer()
                            if callingViewModel.isLocalVideoStreamEnabled {
                                Image(systemName: "video")
                                    .padding()
                            } else {
                                Image(systemName: "video.slash")
                                    .padding()
                            }
                            Spacer()
                        }
                    })
                    Button(action: { callingViewModel.mute() }, label: {
                        HStack {
                            Spacer()
                            if callingViewModel.isMicrophoneMuted {
                                Image(systemName: "speaker.slash")
                                    .padding()
                            } else {
                                Image(systemName: "speaker.wave.2")
                                    .padding()
                            }
                            Spacer()
                        }
                    })
                    Button(action: { callingViewModel.endCall() }, label: {
                        HStack {
                            Spacer()
                            Image(systemName: "phone.down")
                                .foregroundColor(.red)
                                .padding()
                            Spacer()
                        }
                    })
                }
                .font(.largeTitle)
                .padding(.bottom, 5)
            }
            .zIndex(1)

        }
        .ignoresSafeArea(edges: .all)
    }
}

struct GroupCall_Previews: PreviewProvider {
    static var previews: some View {
        GroupCallView()
            .environmentObject(AuthenticationViewModel())
            .environmentObject(CallingViewModel())
    }
}

#### Preview Time

![](img/4.png)
DirectCallView and GroupCallView preview

On direct call, which is 1:1, we are using DirectCallView which has local video and full screen remote video. On the other hand to direct call, we will use GroupCallView for GroupCall which will automatically resize views according to the participants number.

Now let’s change our HomeView.swift as below. It checks whether callAgent is initialized or not and prompt user to sign In if callAgent hasn’t been initialized.

#### [HomeView.swift](code/HomeView.swift)

In [ ]:
import SwiftUI
import AzureCommunicationCalling
import AVFoundation

struct HomeView: View {
    @EnvironmentObject var authenticationViewModel: AuthenticationViewModel
    @EnvironmentObject var callingViewModel: CallingViewModel

    var body: some View {
        VStack(spacing: 16) {
            if callingViewModel.hasCallAgent {
                VStack {
                    Form {
                        Section {
                            HStack(alignment: .top) {
                                TextField("GroupId", text: $callingViewModel.groupId)

                                Button(action: {
                                    callingViewModel.callee = ""
                                }, label: {
                                    Image(systemName: "delete.left")
                                        .foregroundColor(Color(UIColor.opaqueSeparator))
                                })
                            }

                            Button(action: joinGroup) {
                                Text("Join Group")
                            }
                        }

                        Section {
                            HStack(alignment: .top) {
                                TextField("Who would you like to call?", text: $callingViewModel.callee)

                                Button(action: {
                                    callingViewModel.callee = ""
                                }, label: {
                                    Image(systemName: "delete.left")
                                        .foregroundColor(Color(UIColor.opaqueSeparator))
                                })
                            }

                            Button(action: startCall) {
                                Text("Start Call")
                            }

                            Button(action: endCall) {
                                Text("End Call")
                            }
                            Text("Call State: \(callingViewModel.callState.rawValue)")
                        }
                    }
                }
            } else {
                Text("Please sign in and update your display name.")
                Button(action: { authenticationViewModel.currentTab = .profile }, label: {
                    HStack {
                        Spacer()
                        Text("Navigate to signIn page")
                        Spacer()
                    }
                })
            }
        }
        .onReceive(self.authenticationViewModel.$currentTab, perform: { currentTab in
            if (currentTab == .home) {
                print("set signInRequired to false in Home view.\n")
                self.authenticationViewModel.signInRequired = false
            }
        })
    }

    func joinGroup() {
        callingViewModel.joinGroup()
    }

    func startCall() {
        callingViewModel.startCall()
    }

    func endCall() {
        callingViewModel.endCall()
    }
}

struct HomeView_Previews: PreviewProvider {
    static var previews: some View {
        HomeView()
            .environmentObject(AuthenticationViewModel())
            .environmentObject(CallingViewModel())
    }
}

Now we will create a few more views related to profile which include sign in and update profile(user id and tags for notification hub).

Create CommunicationsSettings.swift as below.

#### [CommunicationsSettings.swift](code/CommunicationsSettings.swift)

In [ ]:
import SwiftUI

struct CommunicationsSetting: View {
    @EnvironmentObject var authenticationViewModel: AuthenticationViewModel
    @EnvironmentObject var callingViewModel: CallingViewModel
    @State private var isExpanded: Bool = false

    var body: some View {
        Group {
            Form {
                Section(header: Text("Identifier")) {
                    Text(authenticationViewModel.identifier)
                }

                Section(header: Text("CommunicationTokenCredential")) {
                    DisclosureGroup(isExpanded: $isExpanded.animation(),
                    content: {
                        VStack(alignment: .leading) {
                            Text(authenticationViewModel.token)
                                .frame(height: 100)
                        }
                    }, label: {
                        Text("Token")
                    })
                }

                Section(header: Text("DisplayName")) {
                    HStack(alignment: .top) {
                        TextField("DisplayName", text: $authenticationViewModel.displayName)
                            .autocapitalization(.none)

                        Button(action: {
                            authenticationViewModel.displayName = ""
                        }, label: {
                            Image(systemName: "delete.left")
                                .foregroundColor(Color(UIColor.opaqueSeparator))
                        })
                    }
                }

                Button(action: { initCallAgent() }, label: {
                    HStack {
                        Spacer()
                        Text("Update display name")
                        if authenticationViewModel.isAuthenticating {
                            ProgressView()
                        }
                        Spacer()
                    }
                })

                Section {
                    Button(action: { signOut() }, label: {
                        HStack {
                            Spacer()
                            Text("SignOut")
                            if authenticationViewModel.isAuthenticating {
                                ProgressView()
                            }
                            Spacer()
                        }
                    })
                }
            }
        }
    }

    func initCallAgent() {
        print("Init CallAgent")
        if let communicationUserToken = authenticationViewModel.getCommunicationUserToken() {
            callingViewModel.initCallAgent(communicationUserTokenModel: communicationUserToken, displayName: authenticationViewModel.displayName) { (success) in
                print("callAgent initialized")
                //callingViewModel.registerVoIP()
            }
        } else {
            print("callClient not intialized")
        }
    }

    func signOut() {
        print("Signing out...")
        callingViewModel.resetCallAgent()
        authenticationViewModel.currentTab = .home
    }
}

struct CommunicationsSetting_Previews: PreviewProvider {
    static var previews: some View {
        CommunicationsSetting()
            .environmentObject(AuthenticationViewModel())
            .environmentObject(CallingViewModel())
    }
}

On CommunicationsSetting.swift, you can change your own token or displayName and signOut.


#### Preview
![](img/5.png)
CommunicationsSetting.swift

Create TagsList.swift and TagRow.swift as below. This is need to manage tags for notification hub.

#### [TagsListView.swift](code/TagsListView.swift)

In [ ]:
import SwiftUI

struct TagListView: View {
    var tags: [String]
    var onDelete: (IndexSet) -> Void

    var body: some View {
        List {
            ForEach(tags, id: \.self) {
                Row(title: $0);
            }
            .onDelete(perform: {
                self.onDelete($0)
            })
        }
    }
}

struct TagListView_Previews: PreviewProvider {
    static var previews: some View {
        TagListView(tags: ["tag1", "tag2"], onDelete: {_ in })
    }
}

#### [TagRow.swift](code/TagRow.swift)

In [ ]:
import SwiftUI

struct Row: View {
    var title: String = "<nil_title>"
    var message: String = ""

    var body: some View {
        HStack {
            Text(title)
            .foregroundColor(Color.gray)
            Text(message)
            .foregroundColor(Color.gray)
            Spacer()
        }
    }
}

struct Row_Previews: PreviewProvider {
    static var previews: some View {
        Row()
            .previewLayout(.fixed(width: 300, height: 70))
    }
}

Create NotificationsSetting.swift as below. NotificationsSetting view is where you can set userId and manage tags to receive related notifications from hub.

#### [NotificationsSetting.swift](code/NotificationsSetting.swift)

In [ ]:
import SwiftUI
import WindowsAzureMessaging

struct NotificationsSetting: View {
    @EnvironmentObject var notificationViewModel: NotificationViewModel
    @State var tag: String = "";

    var body: some View {
        VStack(alignment: .leading) {
            Text("Device Token:")
                .font(.headline)
                .padding(.leading)
            Text(notificationViewModel.pushChannel)
                .font(.footnote)
                .foregroundColor(Color.gray)
                .padding([.leading, .bottom, .trailing])

            Text("Installation ID:")
                .font(.headline)
                .padding(.leading)
            Text(notificationViewModel.installationId)
                .font(.footnote)
                .foregroundColor(Color.gray)
                .padding([.leading, .bottom, .trailing])

            Text("User ID:")
                .font(.headline)
                .padding(.leading)
            TextField("Set User ID", text: $notificationViewModel.userId, onEditingChanged: {focus in
                if(!focus) {
                    notificationViewModel.setUserId()
                }
            })
                .textFieldStyle(RoundedBorderTextFieldStyle())
                .padding([.leading, .bottom, .trailing])

            Text("Tags:")
                .font(.headline)
                .padding(.leading)
            TextField("Add new tag", text: $tag, onCommit: {
                if(self.tag != "") {
                    notificationViewModel.addTag(tag: self.tag)
                    self.tag = ""
                }
            })
                .textFieldStyle(RoundedBorderTextFieldStyle())
                .padding([.leading, .bottom, .trailing])

            TagListView(tags: notificationViewModel.tags, onDelete: {
                $0.forEach({
                    MSNotificationHub.removeTag(notificationViewModel.tags.remove(at: $0));
                })
            })

            Spacer()
        }
    }
}

struct NotificationsSetting_Previews: PreviewProvider {
    static var previews: some View {
        NotificationsSetting()
            .environmentObject(NotificationViewModel())
    }
}

#### Preview
![](img/6.png)
NotificationsSetting.swift

Add ProfileCategory enums to Enums folder to navigate between Communications and Notifications settings. Modify Profile.swift as well.

#### [ProfileCategory.swift](code/ProfileCategory.swift)

In [ ]:
enum ProfileCategory: Int, Identifiable, CaseIterable {
    case coummunications
    case notifications

    var id: Int {
        return rawValue
    }
}

#### [ProfileView.swift](code/ProfileView.swift)

In [ ]:
import SwiftUI

struct ProfileView: View {
    @EnvironmentObject var authenticationViewModel: AuthenticationViewModel
    @EnvironmentObject var notificationViewModel: NotificationViewModel
    @EnvironmentObject var callingViewModel: CallingViewModel

    @State var selectedCategory = ProfileCategory.coummunications

    var body: some View {
        VStack {
            if callingViewModel.hasCallAgent {
                Picker("Profile", selection: $selectedCategory) {
                    Text("Communications").tag(ProfileCategory.coummunications)
                    Text("Notifications").tag(ProfileCategory.notifications)
                }
                .pickerStyle(SegmentedPickerStyle())

                if selectedCategory == ProfileCategory.coummunications {
                    CommunicationsSetting()
                        .animation(.easeOut)
                        .transition(.move(edge: .trailing))
                }

                if selectedCategory == ProfileCategory.notifications {
                    NotificationsSetting()
                        .animation(.easeOut)
                        .transition(.move(edge: .leading))
                }
            } else {
                Text("Sign In required")
            }
        }
        .onReceive(authenticationViewModel.$currentTab, perform: { currentTab in
            if (currentTab == .profile && !callingViewModel.hasCallAgent) {
                print("set signInRequired to true in Profile view.\n")
                authenticationViewModel.signInRequired = true
            }
        })
        .onAppear(perform: {
            print("onAppear: Profile view")
        })
    }
}

struct ProfileView_Previews: PreviewProvider {
    static var previews: some View {
        TabView {
            ProfileView()
                .tabItem {
                    Label("Profile", systemImage: "person")
                }
                .environmentObject(AuthenticationViewModel())
                .environmentObject(NotificationViewModel())
                .environmentObject(CallingViewModel())
        }
    }
}

Add SignInView.swift to handle signIn.

#### [SignInView.swift](code/SignInView.swift)

In [ ]:
import SwiftUI
import Combine

struct SignInView: View {
    @Environment(\.presentationMode) var presentationMode: Binding<PresentationMode>
    @EnvironmentObject var authenticationViewModel: AuthenticationViewModel
    @EnvironmentObject var notificationViewModel: NotificationViewModel
    @EnvironmentObject var callingViewModel: CallingViewModel


    var body: some View {
        NavigationView {
            Form {
                Section(header: Text("Sign in with email")) {
                    TextField("Email", text: $authenticationViewModel.email)
                        .textContentType(.emailAddress)
                        .autocapitalization(.none)
                        .cornerRadius(5.0)

                    TextField("Password", text: $authenticationViewModel.password)
                        .textContentType(.password)
                        .autocapitalization(.none)
                        .cornerRadius(5.0)

                    Button(action: { signInWithEmail()}, label: {
                        HStack {
                            Spacer()
                            Text("Sign in with email")
                            Spacer()
                        }
                    })
                }

                Section(header: Text("Sign in with token")) {
                    HStack(alignment: .top) {
                        TextEditor(text: $authenticationViewModel.token)
                            .frame(height: 100, alignment: .leading)

                        Button(action: {
                            authenticationViewModel.token = ""
                        }, label: {
                            Image(systemName: "delete.left")
                                .foregroundColor(Color(UIColor.opaqueSeparator))
                        })
                        .padding(.top, 8)
                    }

                    Button(action: { self.signInWithTokken() }, label: {
                        HStack {
                            Spacer()
                            Text("Sign in with token")
                            Spacer()
                        }
                    })
                }

                Button(action: { self.cancel() }, label: {
                    HStack {
                        Spacer()
                        Text("Cancel")
                        Spacer()
                    }
                })
            }
            .navigationBarTitle("Sign In", displayMode: .inline)
        }
        .onReceive(callingViewModel.$hasCallAgent, perform: { hasCallAgent in
            if hasCallAgent {
                presentationMode.wrappedValue.dismiss()
            }
        })
    }

    func signInWithEmail() {
        print("Sign in with email")
        print("Sign in to your auth server and get ACS token from the api.")
        print("Initialize CommunicationTokenCredential with retrieved token from the auth server.")
    }

    func signInWithTokken() {
        print("Sign in with token")
        if let communicationUserTokenModel = authenticationViewModel.getCommunicationUserToken() {
            callingViewModel.initCallAgent(communicationUserTokenModel: communicationUserTokenModel, displayName: authenticationViewModel.displayName) { success in
                if success {
                    notificationViewModel.connectToHub()
                    print("successfully signed in.\n")
                } else {
                    print("callAgent not intialized.\n")
                }
            }
        }
    }

    func cancel() {
        print("Cancel sign in")
        self.authenticationViewModel.currentTab = .home
    }
}

struct SignInView_Previews: PreviewProvider {
    static var previews: some View {
        SignInView()
            .environmentObject(AuthenticationViewModel())
            .environmentObject(NotificationViewModel())
            .environmentObject(CallingViewModel())
    }
}

Add SheetType.swift enum to Enums folder. This enum will handle whether to show callview or signIn page.

#### [SheetType.swift](code/SheetType.swift)

In [ ]:
enum SheetType: Int, Identifiable, CaseIterable {
    case signInRequired
    case displayNotification
    case callView

    var id: Int {
        return rawValue
    }
}

### And.. last… the ContentView!!!
Modify ContentView.swift as below.

#### [ContentView_edit.swift](code/ContentView_edit.swift)

In [ ]:
import SwiftUI
import WindowsAzureMessaging
import AzureCommunicationCalling

struct ContentView: View {
    @EnvironmentObject var authenticationViewModel: AuthenticationViewModel
    @EnvironmentObject var notificationViewModel: NotificationViewModel
    @EnvironmentObject var callingViewModel: CallingViewModel
    @State private var sheetType: SheetType?
    @State private var showingAlert = false
    @State var notification: MSNotificationHubMessage = MSNotificationHubMessage()

    var body: some View {
        NavigationView {
            TabView(selection: $authenticationViewModel.currentTab) {
                ForEach(Tab.allCases) { tab in
                    tab.presentingView
                        .tabItem { tab.tabItem }
                        .tag(tab)
                }
            }
            .navigationBarTitle(authenticationViewModel.currentTab.name, displayMode: .inline)
        }
        .onReceive(authenticationViewModel.$signInRequired, perform: { signInRequired in
            print("signInRequired state changed to \(signInRequired)\n")
            if signInRequired {
                sheetType = .signInRequired
            }
        })
        .onReceive(callingViewModel.$callState, perform: { callState in
            if callState == .connected {
                self.sheetType = .callView
            } else {
                self.sheetType = .none
            }
        })
        .onReceive(self.notificationViewModel.messageReceived) { (notification) in
            self.didReceivePushNotification(notification: notification, messageTapped: false)
        }
        .onReceive(self.notificationViewModel.messageTapped) { (notification) in
            self.didReceivePushNotification(notification: notification, messageTapped: true)
        }
        .alert(isPresented: $showingAlert) {
            Alert(title: Text(self.notification.title ?? "Important message"), message: Text(self.notification.body ?? "Wear sunscreen"), dismissButton: .default(Text("Got it!")))
        }
        .fullScreenCover(item: $sheetType) { item in
            switch item {
            case .signInRequired:
                SignInView()
                    .environmentObject(authenticationViewModel)
                    .environmentObject(notificationViewModel)
                    .environmentObject(callingViewModel)
            case .callView:
                CallView()
                    .environmentObject(authenticationViewModel)
                    .environmentObject(callingViewModel)
            default:
                Text("Not specified yet!")
            }
        }
        .onAppear(
            perform: checkToken
        )
        .environmentObject(authenticationViewModel)
        .environmentObject(notificationViewModel)
        .environmentObject(callingViewModel)
    }

    func checkToken() {
        if !callingViewModel.hasCallAgent {
            if let communicationUserTokenModel = authenticationViewModel.getCommunicationUserToken() {
                callingViewModel.initCallAgent(communicationUserTokenModel: communicationUserTokenModel, displayName: authenticationViewModel.displayName) { (success) in
                    if success {
                        notificationViewModel.connectToHub()
                    } else {
                        print("callAgent not intialized.\n")
                    }
                }
            } else {
                print("no token found stay at Home.")
            }
        } else {
            notificationViewModel.connectToHub()
        }
    }

    func didReceivePushNotification(notification: Notification, messageTapped: Bool) {
        let message = notification.userInfo!["message"] as! MSNotificationHubMessage
        NSLog("Received notification: %@; %@", message.title ?? "<nil>", message.body)

        // Assign the latest notification to self.notification.
        self.notification = message

        // Display Alert if message is tapped from background.
        if messageTapped {
            self.showingAlert = true
        }
    }
}

struct ContentView_Previews: PreviewProvider {
    static var previews: some View {
        ContentView()
            .environmentObject(AuthenticationViewModel())
            .environmentObject(NotificationViewModel())
            .environmentObject(CallingViewModel())
    }
}

### Conclusion

Make sure the project build and everything looks ok. Run the project on real device. Change resouceName of AzureCommunicationVideoCallingSampleApp to “SecondUser” to deploy onto another device if you have one.

Call each other and see the notification wakes up the device even if it’s on lockscreen or app is terminated.

Thanks for reading this article. Integrating ACS and ANH all together at once was kinda heavy task especially there are not much of sample and documentation around.

Here is ~~my~~ the [github](https://github.com/LuisFX/swift-callkit-pushkit-acs-anh) repository that has all the source code explained above.

(linked to luisfx's fork for updates to original... all credits to @hyounoo and his [original repo](https://github.com/hyounoo/Add-Video-Calling) )

Please be aware that this is more like POC and not Production ready as Azure Communication Services is not on GA yet as of Feb 19th 2021.

Hope you enjoyed my article and wish you had successfully run the application and make and receive some calls.

Thank you!